# Create Apects and Embedding for every Review
* using all_df_filtered dataset
* get the topics(aspect) from reviews (LDA)
* get BERT embedding from reviews
* get aspect score from reviews

# Import

In [4]:
import pandas as pd
import numpy as np
from collections import Counter
import pickle

In [5]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [6]:
RANDOM_STATE = 24

In [7]:
# 導入資料
all_df = pd.read_pickle('../Data/restaurant_only_s_with_embedding.pkl').reset_index(drop=True)
lda_model = pickle.load(open('../data/lda_model_v2.pkl', 'rb'))
docs = pickle.load(open("../data/LDA_training_docs_v2.pkl", "rb"))

In [8]:
print(all_df.shape)
# print(f'Number of unique tokens: {len(dictionary)}')

(24852, 24)


# Review Features

## General Inquirer
* Most reviews have general inquirer 

In [ ]:
# import nltk
# from nltk.corpus import opinion_lexicon

# # Download the General Inquirer lexicon
# nltk.download('opinion_lexicon')
# general_inquirer_words = set(opinion_lexicon.words())

# # Function to check if a string contains General Inquirer words
# def contains_general_inquirer(text):
#     words = set(text.lower().split())
#     common_words = words.intersection(general_inquirer_words)
#     return len(common_words) > 0

In [ ]:
# # unhash if necessary
# cnt = 0
# for idx , row in all_df.iterrows():
#     # if cnt%10000 == 0:
#     #     print(f'Now progress .... {cnt}')
#     if contains_general_inquirer(row.text):
#         pass
#     else:
#         cnt+=1
#         row.text = ''
# all_df.to_pickle('../Data/restaurant_only_filtered.pkl')

## LDA 

In [6]:
import spacy
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel
from gensim import matutils
#python -m spacy download en_core_web_md 

import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

In [7]:
nlp = spacy.load("en_core_web_md")
keep_pos = ['NOUN','ADJ','ADV','VERB']
# removal= ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE', 'NUM', 'SYM']

def preprocess(text : str) -> list:
    
    for summary in nlp.pipe([text]):
        proj_tok = [token.lemma_.lower() for token in summary \
                    if token.pos_ in keep_pos and not token.is_stop and token.is_alpha]

    return proj_tok

In [ ]:
# # unhash if necessary 
# def get_training_docs(all_df):
    
#     docs = []
    
#     for name in Counter(all_df.name):
#         res = all_df[all_df.name == name]
        
#         docs.append(preprocess(''.join(res.text.values.tolist())[:5000]))

#     return docs

# docs = get_training_docs(all_df)
# with open("../data/LDA_training_docs_v2.pkl","wb") as file:
#     # Use pickle.dump() to serialize and save the object to the file
#     pickle.dump(docs, file)

In [ ]:
# # Train LDA

# # Create a dictionary representation of the documents.
# dictionary = Dictionary(docs)

# # Filter out words that occur less than 20 documents, or more than 50% of the documents.
# dictionary.filter_extremes(no_below=50, no_above=0.5,keep_n=1000)

# #### doc2Bow
# corpus = [dictionary.doc2bow(doc) for doc in docs]

# # # building models
# lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=50, \
#                          num_topics=50, workers = 4, passes=10)

# with open('../data/lda_model_v2.pkl', 'wb') as file:
#     # Use pickle.dump() to serialize and save the object to the file
#     pickle.dump(lda_model, file)

In [8]:
# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=50, no_above=0.5,keep_n=1000)

In [9]:
# check topic
lda_model.print_topics(-1)

[(0,
  '0.060*"sandwich" + 0.025*"quick" + 0.019*"meat" + 0.018*"tasty" + 0.017*"wing" + 0.016*"pick" + 0.014*"guy" + 0.013*"delivery" + 0.013*"beef" + 0.012*"different"'),
 (1,
  '0.029*"fry" + 0.029*"table" + 0.029*"potato" + 0.029*"lady" + 0.029*"drive" + 0.021*"tip" + 0.021*"friend" + 0.021*"min" + 0.021*"sit" + 0.021*"bring"'),
 (2,
  '0.077*"breakfast" + 0.030*"coffee" + 0.027*"egg" + 0.026*"toast" + 0.022*"server" + 0.020*"table" + 0.013*"bacon" + 0.012*"feel" + 0.012*"option" + 0.011*"sit"'),
 (3,
  '0.059*"pork" + 0.037*"bean" + 0.029*"sweet" + 0.020*"meat" + 0.018*"cold" + 0.017*"table" + 0.017*"drive" + 0.017*"rice" + 0.017*"sit" + 0.016*"home"'),
 (4,
  '0.043*"rice" + 0.031*"onion" + 0.022*"pepper" + 0.022*"veggie" + 0.021*"quick" + 0.020*"egg" + 0.019*"lot" + 0.019*"fast" + 0.019*"roll" + 0.016*"base"'),
 (5,
  '0.020*"pay" + 0.017*"employee" + 0.016*"manager" + 0.015*"shrimp" + 0.014*"review" + 0.013*"area" + 0.013*"charge" + 0.012*"spicy" + 0.012*"night" + 0.012*"wrong"

## Bert Embedding

In [10]:
from transformers import BertTokenizer, BertModel

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
bert_tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = BertModel.from_pretrained(model_name,output_hidden_states = True).to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Aspect Base

In [6]:
# given aspects
restaurant_aspects = [
    "Food Quality",
    "Technique",
    "Ingredients",
    "Creativity",
    "Consistency",
    "Value for Money",
    "Mastery of Flavors",
    "Presentation",
    "Dining Experience",
    "Ambiance",
    "Wine and Beverage Selection",
    "Service",
    "Attention to Detail",
    "Chef's Vision",
    "Authenticity"
]



In [13]:
# Get the most important words for each topic and turn it into aspects 
num_words = 10  # Number of top words to retrieve for each topic
topics = lda_model.show_topics(num_topics=-1, num_words=num_words, formatted=False)
LDA_aspect_list = []
# Print the most important words for each topic
for topic_id, topic in topics:
    # print(f'Topic #{topic_id} , {topic[0][0]}')
    for word, _ in topic:
        if word not in LDA_aspect_list:
            LDA_aspect_list.append(word)
            break

In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification,DebertaV2Tokenizer

absa_tokenizer = AutoTokenizer.from_pretrained("yangheng/deberta-v3-base-absa-v1.1")
absa_model = AutoModelForSequenceClassification.from_pretrained("yangheng/deberta-v3-base-absa-v1.1").to(device)
# inputs = tokenizer("[CLS] when tables opened up, the manager sat another party before us. [SEP] manager [SEP]", return_tensors="pt")
# outputs = model(**inputs)
# SequenceClassifierOutput(loss=None, logits=tensor([[ 4.3158, -2.3969, -1.7158]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

# Functions

In [11]:
# functions

def get_aspect(text):
    corpus = [dictionary.doc2bow(doc) for doc in [preprocess(text)]]
    return lda_model[corpus][0]

def get_bert_embedding(text):
    encoded_input = bert_tokenizer(text, max_length = 128 , padding = True , truncation = True,  return_tensors='pt')
    encoded_input = encoded_input.to(device)
    with torch.no_grad():
        outputs = bert_model(**encoded_input)
    out_cpu = outputs.hidden_states[-1][:, 0, :].cpu()
    
    del outputs
    
    return out_cpu

def get_LDA_sentiment_score(text , LDA_aspects, aspect_list):
    scores = {}
    for aspect_id, _ in LDA_aspects:
        input_str = "[CLS]"+text+" [SEP]"+str(aspect_list[aspect_id])+"[SEP]"
        inputs = absa_tokenizer(input_str, return_tensors="pt")
        inputs = inputs.to(device)
        with torch.no_grad():
            outputs = absa_model(**inputs)
        scores[aspect_id] = torch.sigmoid(outputs.logits).cpu()
        
        del outputs

    return scores

def get_self_sentiment_score(text , aspect_list):
    scores = []
    for aspect in aspect_list:
        input_str = "[CLS]"+text+" [SEP]"+str(aspect)+"[SEP]"
        inputs = absa_tokenizer(input_str, return_tensors="pt")
        inputs = inputs.to(device)
        with torch.no_grad():
            outputs = absa_model(**inputs)
        scores.append(torch.sigmoid(outputs.logits).cpu())
        
        del outputs

    return scores
    

In [39]:
all_df['LDA_aspects'] = all_df['text'].apply(lambda x : get_aspect(x))

In [42]:
all_df['BERT_embedding'] = all_df['text'].apply(lambda x : get_bert_embedding(x))

In [14]:
all_df['LDA_senti'] = all_df.apply(lambda x : get_LDA_sentiment_score(x.text , x.LDA_aspects , LDA_aspect_list) , axis = 1)

In [9]:
# 2 hrs long
all_df['self_senti'] = all_df.apply(lambda x : get_self_sentiment_score(x.text , restaurant_aspects) , axis = 1)

In [15]:
all_df.to_pickle('../Data/restaurant_only_s_with_embedding.pkl')